In [ ]:
import numpy as np
import math

def floyd_warshall(graph):
    """
    Implements the Floyd-Warshall algorithm for finding shortest paths between all pairs of vertices.

    Args:
        graph: A 2D array representing the adjacency matrix of the graph.
               graph[i][j] = weight of edge from i to j, or float('inf') if no direct edge.
               Diagonal elements (graph[i][i]) should be 0.

    Returns:
        dist: A 2D array where dist[i][j] is the shortest distance from vertex i to vertex j.
        next: A 2D array where next[i][j] is the next vertex on the shortest path from i to j.
        iterations: List of distance matrices after each iteration k.
    """
    n = len(graph)

    # Initialize distance matrix and next vertex matrix
    dist = np.copy(graph)
    next = [[None for _ in range(n)] for _ in range(n)]

    # Initialize the next matrix
    for i in range(n):
        for j in range(n):
            if i != j and dist[i][j] < float('inf'):
                next[i][j] = j

    # Store iterations for visualization
    iterations = [np.copy(dist)]

    # Main algorithm: consider each vertex as an intermediate point
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if dist[i][k] != float('inf') and dist[k][j] != float('inf'):
                    if dist[i][j] > dist[i][k] + dist[k][j]:
                        dist[i][j] = dist[i][k] + dist[k][j]
                        next[i][j] = next[i][k]

        # Store the matrix after considering vertex k as intermediate
        iterations.append(np.copy(dist))

    return dist, next, iterations

def reconstruct_path(next, u, v):
    """
    Reconstructs the shortest path from vertex u to vertex v.

    Args:
        next: The next vertex matrix from floyd_warshall.
        u: The start vertex.
        v: The end vertex.

    Returns:
        path: A list of vertices representing the shortest path from u to v.
    """
    if next[u][v] is None:
        return []  # No path exists

    path = [u]
    while u != v:
        u = next[u][v]
        path.append(u)

    return path

def print_matrix(matrix, header=None):
    """
    Prints a matrix in a readable format.

    Args:
        matrix: The matrix to print.
        header: Optional title to print before the matrix.
    """
    if header:
        print(f"\n{header}")

    for row in matrix:
        formatted_row = [str(int(x)) if x != float('inf') else "∞" for x in row]
        print(" ".join(f"{x:>3}" for x in formatted_row))

# Define the graph from the image (using 0-indexed vertices)
def create_graph_from_image():
    # Number of vertices (1-5 in the image, but we'll use 0-4)
    n = 5

    # Initialize with all edges as infinity
    graph = np.full((n, n), float('inf'))

    # Set diagonal elements to 0
    np.fill_diagonal(graph, 0)

    # Add the edges from the image (converting from 1-indexed to 0-indexed)
    edges = [
        (0, 1, 3),   # 1 -> 2 with weight 3
        (0, 2, 8),   # 1 -> 3 with weight 8
        (0, 4, -4),  # 1 -> 5 with weight -4
        (1, 2, 4),   # 2 -> 3 with weight 4
        (1, 3, 1),   # 2 -> 4 with weight 1
        (1, 4, 7),   # 2 -> 5 with weight 7
        (2, 1, -5),  # 3 -> 2 with weight -5
        (3, 0, 2),   # 4 -> 1 with weight 2
        (3, 2, -5),  # 4 -> 3 with weight -5
        (4, 1, 7),   # 5 -> 2 with weight 7
        (4, 3, 6)    # 5 -> 4 with weight 6
    ]

    for u, v, w in edges:
        graph[u][v] = w

    return graph

# Create the graph from the image
graph = create_graph_from_image()

# Run Floyd-Warshall algorithm
final_dist, next_vertex, iterations = floyd_warshall(graph)

# Print the original graph
print("Original Graph (Adjacency Matrix):")
print_matrix(graph)

# Print all intermediate matrices
for k, matrix in enumerate(iterations):
    if k == 0:
        print_matrix(matrix, f"\nInitial Distance Matrix (L^(0)):")
    else:
        print_matrix(matrix, f"\nDistance Matrix after considering vertex {k} as intermediate (L^({k})):")

# Verify the property mentioned in the image: L^(n) = L^(n-1) for n >= 5
n = len(graph)
is_equal = np.array_equal(iterations[n], iterations[n-1])
print(f"\nVerifying L^({n}) = L^({n-1}): {is_equal}")

# Print some example shortest paths
print("\nShortest paths between vertices:")
for i in range(n):
    for j in range(n):
        if i != j:
            path = reconstruct_path(next_vertex, i, j)
            # Convert to 1-indexed for better readability
            path_1indexed = [v + 1 for v in path]
            dist = final_dist[i][j]
            print(f"From {i+1} to {j+1}: Path = {path_1indexed}, Distance = {dist}")

# Examine if there are any negative cycles
has_negative_cycle = False
for i in range(n):
    if final_dist[i][i] < 0:
        has_negative_cycle = True
        break

print(f"\nGraph contains negative cycle: {has_negative_cycle}")

Original Graph (Adjacency Matrix):
  0   3   8   ∞  -4
  ∞   0   4   1   7
  ∞  -5   0   ∞   ∞
  2   ∞  -5   0   ∞
  ∞   7   ∞   6   0


Initial Distance Matrix (L^(0)):
  0   3   8   ∞  -4
  ∞   0   4   1   7
  ∞  -5   0   ∞   ∞
  2   ∞  -5   0   ∞
  ∞   7   ∞   6   0


Distance Matrix after considering vertex 1 as intermediate (L^(1)):
  0   3   8   ∞  -4
  ∞   0   4   1   7
  ∞  -5   0   ∞   ∞
  2   5  -5   0  -2
  ∞   7   ∞   6   0


Distance Matrix after considering vertex 2 as intermediate (L^(2)):
  0   3   7   4  -4
  ∞   0   4   1   7
  ∞  -5  -1  -4   2
  2   5  -5   0  -2
  ∞   7  11   6   0


Distance Matrix after considering vertex 3 as intermediate (L^(3)):
  0   2   6   2  -4
  ∞  -1   3  -1   5
  ∞  -6  -2  -6   0
  2 -11  -7 -13  -7
  ∞   5   9   3   0


Distance Matrix after considering vertex 4 as intermediate (L^(4)):
  0  -9  -5 -11 -18
  1 -12  -8 -14 -21
 -4 -17 -13 -19 -26
-11 -24 -20 -26 -33
 -8 -21 -17 -23 -56


Distance Matrix after considering vertex 5 as in